In [24]:
import pandas as pd
import numpy as np

In [25]:
file='OT_trader_input_format5.xlsx'

In [26]:
# filenames=['Time Buckets','TC Curve','fwprices','Base_discounts','Top-off_discounts','Base_cargo_specs','Top-off_cargo_specs','Parcel_calc_specs']

In [27]:
# timee_buc

In [28]:
# tc_cur

In [29]:
# fc_cur

In [30]:
# base_disc

In [31]:
# top_disc

In [32]:
# base_spec

In [33]:
# top_spec

In [ ]:
# parc_spec

In [15]:
# filenamessavedas=[timee_buc,tc_cur,fc_cur,base_disc,top_disc,base_spec,top_spec,parc_spec]

In [16]:
# for i in range(len(filenames)):
#     filenamessavedas[i]=pd.read_excel(file,sheet_name=filenames[i])

In [34]:
time_buc =pd.read_excel(file,sheet_name='Time Buckets')
tc_cur =pd.read_excel(file,sheet_name='TC Curve')
fc_cur =pd.read_excel(file,sheet_name='fwprices')
base_disc =pd.read_excel(file,sheet_name='Base_discounts')
top_disc =pd.read_excel(file,sheet_name='Top-off_discounts')
base_spec =pd.read_excel(file,sheet_name='Base_cargo_specs')
top_spec =pd.read_excel(file,sheet_name='Top-off_cargo_specs')
parc_spec =pd.read_excel(file,sheet_name='Parcel_calc_specs')

In [35]:
time_buc

,time_bucket,relative_time_bucket_designation,absolute_time_bucket_start
0,T1,R1,2022-12-01
1,T2,R2,2023-01-01
2,T3,R3,2023-02-01
3,T4,R4,2023-03-01
4,T5,R5,2023-04-01


In [36]:
base_disc

,Time,base_discount_1,base_discount_2
0,2022-12-01,-1,-1
1,2023-01-01,-1,-1
2,2023-02-01,-1,-1
3,2023-03-01,-1,-1
4,2023-04-01,-1,-1


In [11]:
def data_process(time,tc_cur,fc_cur,base_disc,top_disc):
    '''Definition: generates 2 imos data for calculation and matching, and retrieves time and fuel curves for time buckets
    Parameters:raw imos data, time buckets, tc curve, fuel curve
    Returns:imos_calc,imos_match, and tc curve and fuel curve for specified time bucket '''
    
    if len(time['absolute_time_bucket_start'].tolist())!=len(set(time['absolute_time_bucket_start'].tolist())):
        print("Dropping duplicates in time buckets")                                             
        time=time.drop_duplicates(subset=['absolute_time_bucket_start'])
    
    if len(tc_cur['Time'].tolist())!=len(set(tc_cur['Time'].tolist())):                                                     
        print("Dropping duplicates in time buckets of tc curve") 
        tc_cur=tc_cur.drop_duplicates(subset=['Time'])
                                                             
    if len(fc_cur['Month'].tolist())!=len(set(fc_cur['Month'].tolist())):     
        print("Dropping duplicates in time buckets of fuel curve")
        fc_cur=fc_cur.drop_duplicates(subset=['Month']) 
                                                         
                                                             
    if len(base_disc['Time'].tolist())!=len(set(base_disc['Time'].tolist())):
        print("Dropping duplicates in time buckets of base discount")                                                              
        base_disc=base_disc.drop_duplicates(subset=['Time'])
                                                             
    if  len(top_disc['Time'].tolist())!=len(set(top_disc['Time'].tolist())):
        print("Dropping duplicates in time buckets of top-off discount")  
        top_disc=top_disc.drop_duplicates(subset=['Time'])
    
    if len(tc_cur.transpose().reset_index()['index'].tolist())!=len(set(tc_cur.transpose().reset_index()['index'].tolist())):                                                     
        print("Dropping duplicates in curve selection of tc curve")                                                         
                                                             
                                                             
    #mapping time bucket and tc curve
    tc_time=pd.merge(time[['absolute_time_bucket_start']],tc_cur,left_on='absolute_time_bucket_start',right_on='Time',how='left')
    tc_time.drop(columns=['absolute_time_bucket_start'],inplace=True)
    #dataframe with fuel curves for specified time buckets
    fc_time=pd.merge(time[['absolute_time_bucket_start']],fc_cur,left_on='absolute_time_bucket_start',right_on='Month',how='left')
    price_dates=time['absolute_time_bucket_start'].dt.strftime('%d-%m-%Y').values.tolist()
    base_disc=pd.merge(time[['absolute_time_bucket_start']],base_disc,left_on='absolute_time_bucket_start',right_on='Time',how='left')
    base_disc.drop(columns=['absolute_time_bucket_start'],inplace=True)
    top_disc=pd.merge(time[['absolute_time_bucket_start']],top_disc,left_on='absolute_time_bucket_start',right_on='Time',how='left')
    top_disc.drop(columns=['absolute_time_bucket_start'],inplace=True)
    return(tc_time,fc_time,price_dates,base_disc,top_disc)

In [12]:
def specs_preproc(df,idtype):
    ''' definition:pre process specifications sheet to remove the merged multilevel headers
        parameters:specification df, type of id='Base cargo', 'Top-off cargo' or 'Parcel calc'
        return:Specification df single level header '''
    new_header=df.iloc[0]
    df=df.iloc[1:]
    df.columns=new_header
    
    var='%s IMOS estimate'%idtype
    df=df.drop_duplicates(subset=[var])
    df.reset_index(drop=True,inplace=True)
    return (df)

In [13]:
tc_time,fc_time,price_dates,base_disc,top_disc=data_process(time_buc,tc_cur,fc_cur,base_disc,top_disc)

Dropping duplicates in time buckets
Dropping duplicates in time buckets of tc curve
Dropping duplicates in time buckets of fuel curve
Dropping duplicates in time buckets of base discount
Dropping duplicates in time buckets of top-off discount


In [ ]:
base_spec=specs_preproc(base_spec,'Base cargo')
top_spec=specs_preproc(top_spec,'Top-off cargo')
parc_spec=specs_preproc(parc_spec,'Parcel calc')